In [1]:
import struct
from os.path import exists
from os.path import getsize
import csv
import progressbar

channels = {b'B':"Battery Current (A)", b'P': "Air Pressure (P)", b'H':"Relative Humidity (percent)", b'M':"Battery Temperature (C)", b'N':"Internal Temperature (C)", b'T':"External Temperature (C)", b'1':"X Acceleration (g)", b'2':"Y Acceleration (g)", b'3':"Z Acceleration (g)", b'4':"X Mangetometer (uT)", b'5':"Y Mangetometer (uT)", b'6':"Z Mangetometer (uT)", b'7':"X Gyro (deg per sec)", b'8':"Y Gyro (deg per sec)", b'9':"Z Gyro (deg per sec)", b'L':"Latitude", b'O':"Longitude", b'A':"Altitude (m)", b'S':"Speed (m per s)", b'I':"Time (seconds)", b'V':"VOC density (ppb)", b'C':"eCO2 density (ppm)"} 
print("file path?")
filepath = input();
print("file name?")
filename = input();
targetfile = filepath + filename
file = open(targetfile, "rb")

file path?
.data2/
file name?
LOG2


In [2]:
n_cycles = int(getsize(targetfile)/9)
current_cycle = 1

with progressbar.ProgressBar(max_value=n_cycles) as bar:    
    dataIn = file.read(1)
    while dataIn:
        try: # Check if the dataIn is a valid data source
            title = channels[dataIn]
        except: # If not, go to the next byte
            print(dataIn)
            dataIn = file.read(1)
            continue
    
        try: # Try to read the data in to process
            byteLong  = file.read(4)
            byteFloat = file.read(4)
        except: # If it fails, we have probably reached EOF; break
            print("Failed to read bytes")
            dataIn = file.read(1)
            break

        # Convert byteLong and byteFloat to an unsigned long and float respectively
        [timestamp] = struct.unpack("L", byteLong)
        [data] = struct.unpack("f", byteFloat)

        # Create CSV file name
        csvfilename = filepath + title + ".csv"
        # Check if it already exists
        # If it doesn't exist, create the file and write the header
        if (not exists(csvfilename)):
            print("New file: " + csvfilename)
            f = open(csvfilename, "a")
            f.write("Time (ms), " + title + "\n")
            f.close()

        # Now that we know it exists and has a header, open it and write the data
        with open(csvfilename, 'a', newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            row = [str(timestamp), str(data)]
            writer.writerow(row)
            
        # Update the progress bar
        current_cycle += 1
        current_cycle = min(current_cycle, n_cycles) # prevent rounding errors from causing problems
        bar.update(current_cycle)

print("done")

 11% (126 of 1101) |##                   | Elapsed Time: 0:00:00 ETA:   0:00:00

New file: .data2/Battery Current (A).csv
New file: .data2/Air Pressure (P).csv
New file: .data2/Relative Humidity (percent).csv
New file: .data2/Battery Temperature (C).csv
New file: .data2/Internal Temperature (C).csv
New file: .data2/External Temperature (C).csv
New file: .data2/X Acceleration (g).csv
New file: .data2/Y Acceleration (g).csv
New file: .data2/Z Acceleration (g).csv
New file: .data2/X Mangetometer (uT).csv
New file: .data2/Y Mangetometer (uT).csv
New file: .data2/Z Mangetometer (uT).csv
New file: .data2/X Gyro (deg per sec).csv
New file: .data2/Y Gyro (deg per sec).csv
New file: .data2/Z Gyro (deg per sec).csv
New file: .data2/Latitude.csv
New file: .data2/Longitude.csv
New file: .data2/VOC density (ppb).csv
New file: .data2/eCO2 density (ppm).csv
New file: .data2/Altitude (m).csv
New file: .data2/Speed (m per s).csv
New file: .data2/Time (seconds).csv


 93% (1032 of 1101) |##################  | Elapsed Time: 0:00:00 ETA:   0:00:00


ValueError: Value 1102 is out of range, should be between 0 and 1101